### Exercise 2.1 Removing outliers from the feature space.
Find a way to remove outliers from your feature space. A simple approach: Compute the standard deviation and consider all values that are more than 3 sigma from the mean as outliers. Note: Assumes normal distribution of the data!

In [ ]:
# Set the input directory where the image files exist
base_dir = '/home2/s183990/jupyter_notebooks/'
input_dir = base_dir + 'pythonII-imaging/images/'

In [ ]:
# Make a list of files in the directory
from os import listdir
filelist = listdir(input_dir)
# print the file list
filelist

In [ ]:
# Sort the filelist 
filelist.sort()
filelist

In [ ]:
# Collect the file names only for files that are tifs: Elegant solution
image_names = [filename for filename in filelist if filename[-4:]=='.tif'] 

In [ ]:
# Check that you have the right files in tiflist. 
print('Found', len(image_names), 'tif files in ', input_dir, ': ')
for file in image_names:
    print(file)

In [ ]:
# Read the images and store into a list
import skimage.io as io # Image file manipulation module
import os #
image_data = [io.imread(os.path.join(input_dir,image)) for image in image_names]

In [ ]:
# Slicing: We only work on one channel for segmentation
image = image_data[0][1,:,:]

In [ ]:
# Gaussian smoothing
import scipy.ndimage as ndi # Image processing package
sigma = 3                                                # Smoothing factor for Gaussian
image_smooth = ndi.filters.gaussian_filter(image,sigma) # Perform smoothing

In [ ]:
# Create an adaptive background
import numpy as np # Array manipulation package
struct = ((np.mgrid[:31,:31][0] - 15)**2 + (np.mgrid[:31,:31][1] - 15)**2) <= 15**2  # Create a disk-shaped structural element
from skimage.filters import rank            # Import module containing mean filter function
bg = rank.mean(image_smooth, selem=struct) # Run a mean filter over the image using the disc

In [ ]:
# Threshold using created background
image_mem = image_smooth >= bg

# Clean by morphological hole filling
image_mem = ndi.binary_fill_holes(np.logical_not(image_mem))

In [ ]:
# Distance transform on thresholded membranes
image_dt= ndi.distance_transform_edt(image_mem)

# Dilating (maximum filter) of distance transform improves results
image_dt = ndi.filters.maximum_filter(image_dt,size=10) 

In [ ]:
# Retrieve and label the local maxima
from skimage.feature import peak_local_max
image_max = peak_local_max(image_dt,indices=False,min_distance=10)  # Local maximum detection
image_max = ndi.label(image_max)[0] # Labeling

In [ ]:
# Get the watershed function and run it
from skimage.morphology import watershed
image_ws = watershed(image_smooth,image_max)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image_ws, cmap='gray')
plt.title('Watershed segmentation')
plt.show()

In [ ]:
# IDENTIFICATION OF CELL EDGES
    
# Define the edge detection function
def edge_finder(footprint_values):
    if (footprint_values == footprint_values[0]).all():
        return 0
    else:
        return 1
        
# Iterate the edge finder over the segmentation
image_edges = ndi.filters.generic_filter(image_ws,edge_finder,size=3)

In [ ]:
# POSTPROCESSING: REMOVING CELLS AT THE IMAGE BORDER    
    
# Create a mask for the image boundary pixels
boundary_mask = np.ones_like(image_ws)   # Initialize with all ones
boundary_mask[1:-1,1:-1] = 0 # Set middle square to 0

In [ ]:
# Iterate over all cells in the segmentation
current_label = 1
for cell_id in np.unique(image_ws):
    
    # If the current cell touches the boundary, remove it
    if np.sum((image_ws==cell_id)*boundary_mask) != 0:
        image_ws[image_ws==cell_id] = 0
            
        # This is to keep the labeling continuous, which is cleaner
    else:
        image_ws[image_ws==cell_id] = current_label
        current_label += 1

In [ ]:
plt.imshow(image_ws, cmap='gray')
plt.title('Watershed segmentation after cleaning')
plt.show()

In [ ]:
# Some frequently used variables
labels = np.unique(image_ws)[1:]  # Labels of cells in segmentation
N = len(labels) # Number of cells in segmentation

In [ ]:
# Initialize feature space
fspace = np.zeros((N,3),dtype=np.float)

In [ ]:
# Get the centroids
cen = np.zeros((N,2))
for index,label in enumerate(labels):
    cen[index,:] = ndi.measurements.center_of_mass(image_ws==label)

In [ ]:
### CELL SIZE (as approximated by segmentation area)

for index,label in enumerate(labels):  
    # Count the number of pixels that are part of that object
    fspace[index,0] = np.sum(image_ws==label)

In [ ]:
### STANDARD DEVIATION OF MEMBRANE INTENSITY

for index,label in enumerate(labels): 
    # Dilation provides yet another way of detecting edge pixels
    edge = ndi.maximum_filter(image_ws!=label,size=3)
    edge[image_ws!=label] = 0
    
    # Stdev of intensity at the edge
    fspace[index,1] = np.std(image[edge==1])

In [ ]:
### INTENSITY ASYMMETRY as the distance of the intensity-weighted center of
#   mass from the unweighted centroid

from scipy.spatial.distance import pdist

for index,label in enumerate(labels):
    
    # Intensity-weighted center of mass
    int_cen = ndi.measurements.center_of_mass(np.ma.array(image,mask=image_ws!=label))
    
    # Distance between the two centroids
    # Note: Below I use the pairwise distance function pdist from scipy's
    #       spatial package. This function is useful for a lot of things in
    #       data analsis, but to be more explicit one could also calculate the
    #       distance between two centroids using simple Pythagoras:
    #           diff = int_cen - cen[index,:]
    #           dist = np.sqrt(np.square(diff[0]) + np.square(diff[1]))
    
    fspace[index,2] = pdist([int_cen,cen[index,:]])[0]

In [ ]:
# Visualize the fspace
plt.boxplot(fspace,labels=["CELL SIZE","STDEV OF MEM INT", "INT ASYM"])
plt.show()

In [ ]:
# EXERCISE 2.1: Find a way to remove outliers from your feature space.

# Find outliers


# Remove outliers (whilst preserving mapping of fspace to labels)


### Exercise 2.2. Normalize  the feature space
Standardize, normalize and/or whiten your feature space as you deem fit, either by transforming the data yourself or using a module function. At the end, visualize the data and compate with the `fspace` boxplot in the previous section to see the effect of the normalization.

In [ ]:
# EXERCISE 2.2:  Standardize, normalize and/or whiten your feature space as you deem fit,
# either by transforming the data yourself or using a module function.

# Center the data around the mean
 

# Whiten
from scipy.cluster.vq import whiten # Hint



# Note: Don't forget to visualize your data again!
